# GPT Action Library: Sharepoint（データ分析/文書要約のためのファイル取得）

## はじめに

このページでは、特定のアプリケーション向けのGPT Actionを構築する開発者向けの手順とガイドを提供します。進める前に、まず以下の情報を理解しておくことを確認してください：
- [GPT Actionsの概要](https://platform.openai.com/docs/actions)
- [GPT Actionsライブラリの概要](https://platform.openai.com/docs/actions/actions-library)
- [GPT Actionをゼロから構築する例](https://platform.openai.com/docs/actions/getting-started)

このソリューションは、Microsoft Graph API の[検索機能](https://learn.microsoft.com/en-us/graph/api/resources/search-api-overview?view=graph-rest-1.0)と[ファイル取得](https://learn.microsoft.com/en-us/graph/api/driveitem-get?view=graph-rest-1.0\&tabs=http)機能を使用して、ユーザーがSharePointやOffice365でアクセス可能なファイルのコンテキストでユーザーの質問に回答するGPTアクションを実現します。Azure Functionsを使用してGraph APIのレスポンスを処理し、人間が読みやすい形式に変換したり、ChatGPTが理解できる構造に整理したりします。このコードは方向性を示すものであり、要件に応じて修正する必要があります。

このソリューションは、[Actionsでファイルを取得](https://platform.openai.com/docs/actions/sending-files)し、会話に直接アップロードしたかのように使用する機能を利用しています。Azure FunctionはChatGPTがファイルに変換するbase64文字列を返します。このソリューションは構造化データと非構造化データの両方を処理できますが、サイズ容量の制限があります（詳細は[こちら](https://platform.openai.com/docs/actions/sending-files)のドキュメントを参照）。

### 価値 + ビジネス活用事例

**価値**: ユーザーはChatGPTの自然言語機能を活用して、SharePoint内のファイルに直接接続できるようになりました

**使用例**: 
- ユーザーが特定のトピックに関連するファイルを検索する必要がある場合
- ユーザーが文書の奥深くに埋もれている重要な質問への回答を必要とする場合

## アーキテクチャ / 例

![](../../../images/solution_1.gif)

このソリューションは、ログインしたユーザーに基づいて、Node.js Azure Functionを使用して以下を実行します：

1. ユーザーの初期質問に基づいて、ユーザーがアクセス権を持つ関連ファイルを検索します。

2. 見つかった各ファイルをbase64文字列に変換します。

3. ChatGPTが期待する構造でデータをフォーマットします（[こちら](https://platform.openai.com/docs/actions/sending-files/inline-option)を参照）。

4. それをChatGPTに返します。GPTは、会話にアップロードしたかのようにそれらのファイルを使用できます。

![](../../../images/solution_1_architecture.png)

## アプリケーション情報

### アプリケーションキーリンク

開始する前に、アプリケーションの以下のリンクをご確認ください：
- アプリケーションWebサイト: https://www.microsoft.com/en-us/microsoft-365/sharepoint/collaboration
- アプリケーションAPI Documentation: https://learn.microsoft.com/en-us/previous-versions/office/developer/sharepoint-rest-reference/

### アプリケーションの前提条件

開始する前に、アプリケーション環境で以下の手順を実行していることを確認してください：
- Sharepoint環境へのアクセス
- Postman（およびAPIとOAuthの知識）

## ミドルウェア情報

[検索概念ファイルガイド](https://learn.microsoft.com/en-us/graph/search-concept-files)に従う場合、[Microsoft Graph Search API](https://learn.microsoft.com/en-us/graph/search-concept-files)は条件に適合するファイルへの参照を返しますが、ファイルの内容自体は返しません。そのため、MSFTエンドポイントに直接アクセスするのではなく、ミドルウェアが必要になります。

そのAPIからのレスポンスを、[こちら](https://platform.openai.com/docs/actions/getting-started/inline-option)で説明されている`openaiFileResponse`の期待される構造に一致するように再構築する必要があります。

### 追加手順

#### Azure Functionのセットアップ

1. [Azure Function クックブック](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_middleware_azure_function)の手順に従ってAzure Functionを設定する

#### 関数コードの追加

認証されたAzure Functionが準備できたので、SharePoint / O365を検索するように関数を更新できます。

2. テスト関数に移動し、[このファイル](https://github.com/openai/openai-cookbook/blob/main/examples/chatgpt/sharepoint_azure_function/solution_one_file_retrieval.js)のコードを貼り付けます。関数を保存してください。

> **このコードは方向性を示すためのものです** - そのまま動作するはずですが、あなたのニーズに合わせてカスタマイズするように設計されています（このドキュメントの最後にある例を参照してください）。

3. **Settings**の下の左側にある**Configuration**タブに移動して、以下の環境変数を設定します。Azure UIによっては、これが**Environment Variables**に直接表示される場合があります。

    1. `TENANT_ID`: 前のセクションからコピー

    2. `CLIENT_ID`: 前のセクションからコピー

4. **Development Tools**の下の**Console**タブに移動します

    1. コンソールで以下のパッケージをインストールします

       1. `npm install @microsoft/microsoft-graph-client`

       2. `npm install axios`

5. これが完了したら、Postmanから再度関数を呼び出し（POST呼び出し）、以下をボディに入力してテストしてください（レスポンスが生成されると思われるクエリと検索語を使用）。

     ```json
    {
        "searchTerm": "<検索語を選択>"
    }
    ```

6. レスポンスが得られた場合、Custom GPTでこれを設定する準備ができています！これを設定する詳細については、Azure Function ページのChatGPTセクションを参照してください。

## より詳細なウォークスルー

以下では、このソリューション固有のセットアップ手順とウォークスルーについて説明します。完全なコードは[こちら](https://github.com/openai/openai-cookbook/blob/main/examples/chatgpt/sharepoint_azure_function/solution_one_file_retrieval.js)で確認できます。

### コードウォークスルー

以下では、関数の異なる部分について説明します。開始する前に、必要なパッケージがインストールされ、環境変数が設定されていることを確認してください（インストール手順のセクションを参照）。

#### 認証の実装

以下では、関数内で使用するいくつかのヘルパー関数を紹介します。

##### Microsoft Graph Clientの初期化

アクセストークンを使用してGraphクライアントを初期化する関数を作成します。これはOffice 365とSharePointを検索するために使用されます。

```javascript
const { Client } = require('@microsoft/microsoft-graph-client');

function initGraphClient(accessToken) {
    return Client.init({
        authProvider: (done) => {
            done(null, accessToken);
        }
    });
}
```

##### On-Behalf-Of (OBO) トークンの取得

この関数は、既存のベアラートークンを使用してMicrosoftのアイデンティティプラットフォームからOBOトークンを要求します。これにより、認証情報を渡すことができ、検索がログインユーザーがアクセスできるファイルのみを返すことが保証されます。

```javascript
const axios = require('axios');
const qs = require('querystring');

async function getOboToken(userAccessToken) {
    const { TENANT_ID, CLIENT_ID, MICROSOFT_PROVIDER_AUTHENTICATION_SECRET } = process.env;
    const params = {
        client_id: CLIENT_ID,
        client_secret: MICROSOFT_PROVIDER_AUTHENTICATION_SECRET,
        grant_type: 'urn:ietf:params:oauth:grant-type:jwt-bearer',
        assertion: userAccessToken,
        requested_token_use: 'on_behalf_of',
        scope: 'https://graph.microsoft.com/.default'
    };

    const url = `https\://login.microsoftonline.com/${TENANT_ID}/oauth2/v2.0/token`;
    try {
        const response = await axios.post(url, qs.stringify(params), {
            headers: { 'Content-Type': 'application/x-www-form-urlencoded' }
        });
        return response.data.access\_token;
    } catch (error) {
        console.error('Error obtaining OBO token:', error.response?.data || error.message);
        throw error;
    }
}
```

#### O365 / SharePointアイテムからのコンテンツ取得

この関数は、ドライブアイテムのコンテンツを取得し、base64文字列に変換して、`openaiFileResponse`形式に合わせて再構築します。

```javascript
const getDriveItemContent = async (client, driveId, itemId, name) => {
   try
       const filePath = `/drives/${driveId}/items/${itemId}`;
       const downloadPath = filePath + `/content`
       // ここでコンテンツを取得してbase64に変換します
       const fileStream = await client.api(downloadPath).getStream();
       let chunks = [];
           for await (let chunk of fileStream) {
               chunks.push(chunk);
           }
       const base64String = Buffer.concat(chunks).toString('base64');
       // ここでレスポンスに含める他のメタデータを取得します
       const file = await client.api(filePath).get();
       const mime_type = file.file.mimeType;
       const name = file.name;
       return {"name":name, "mime_type":mime_type, "content":base64String}
   } catch (error) {
       console.error('Error fetching drive content:', error);
       throw new Error(`Failed to fetch content for ${name}: ${error.message}`);
   }
```

#### リクエストを処理するAzure Functionの作成

これらのヘルパー関数がすべて揃ったので、Azure Functionは、ユーザーを認証し、検索を実行し、検索結果を反復処理してテキストを抽出し、GPTに関連するテキスト部分を取得することで、フローを調整します。

**HTTPリクエストの処理：** 関数は、HTTPリクエストからqueryとsearchTermを抽出することから始まります。Authorizationヘッダーが存在するかチェックし、ベアラートークンを抽出します。

**認証：** ベアラートークンを使用して、上記で定義したgetOboTokenを使用してMicrosoftのアイデンティティプラットフォームからOBOトークンを取得します。

**Graph Clientの初期化：** OBOトークンを使用して、上記で定義したinitGraphClientを使用してMicrosoft Graphクライアントを初期化します。

**ドキュメント検索：** 検索クエリを構築し、Microsoft Graph APIに送信してsearchTermに基づいてドキュメントを検索します。

**ドキュメント処理：** 検索によって返された各ドキュメントに対して：

- getDriveItemContentを使用してドキュメントコンテンツを取得します。

- ドキュメントをbase64文字列に変換し、`openaiFileResponse`構造に合わせて再構築します。

**レスポンス：** 関数はそれらをHTTPレスポンスで送り返します。

```javascript
module.exports = async function (context, req) {
   // const query = req.query.query || (req.body && req.body.query);
   const searchTerm = req.query.searchTerm || (req.body && req.body.searchTerm);
   if (!req.headers.authorization) {
       context.res = {
           status: 400,
           body: 'Authorization header is missing'
       };
       return;
   }
   /// 以下は関数に渡されたトークンを取得し、OBOトークンを取得するために使用します。
   const bearerToken = req.headers.authorization.split(' ')[1];
   let accessToken;
   try {
       accessToken = await getOboToken(bearerToken);
   } catch (error) {
       context.res = {
           status: 500,
           body: `Failed to obtain OBO token: ${error.message}`
       };
       return;
   }
   // 上記で定義したinitGraphClient関数を使用してGraph Clientを初期化します
   let client = initGraphClient(accessToken);
   // これはMicrosoft Graph Search APIで使用される検索ボディです: https://learn.microsoft.com/en-us/graph/search-concept-files
   const requestBody = {
       requests: [
           {
               entityTypes: ['driveItem'],
               query: {
                   queryString: searchTerm
               },
               from: 0,
               // 以下はGraph APIからの上位10件の検索結果を要約するように設定されていますが、ドキュメントに基づいて設定できます。
               size: 10
           }
       ]
   };


   try {
       // ここで検索を実行します
       const list = await client.api('/search/query').post(requestBody);
       const processList = async () => {
           // これは各検索レスポンスを処理し、ファイルの内容を取得してgpt-3.5-turboで要約します
           const results = [];
           await Promise.all(list.value[0].hitsContainers.map(async (container) => {
               for (const hit of container.hits) {
                   if (hit.resource["@odata.type"] === "#microsoft.graph.driveItem") {
                       const { name, id } = hit.resource;
                       // 以下はファイルが存在する場所です
                       const driveId = hit.resource.parentReference.driveId;
                       // 上記で定義したヘルパー関数を使用してコンテンツを取得し、base64に変換し、再構築します
                       const contents = await getDriveItemContent(client, driveId, id, name);
                       results.push(contents)
               }
           }));
           return results;
       };
       let results;
       if (list.value[0].hitsContainers[0].total == 0) {
           // Microsoft Graph APIが結果を返さない場合、APIに結果が見つからないことを返します
           results = 'No results found';
       } else {
           // Microsoft Graph APIが結果を返す場合、processListを実行して反復処理します。
           results = await processList();
           // ここでChatGPTがファイルであることを認識できるようにレスポンスを構造化します
           results = {'openaiFileResponse': results}
       }
       context.res = {
           status: 200,
           body: results
       };
   } catch (error) {
       context.res = {
           status: 500,
           body: `Error performing search or processing results: ${error.message}`,
       };
   }
};
```

### カスタマイズ

以下は、カスタマイズ可能な潜在的な領域です。

- 何も見つからない場合に一定回数再検索するようにGPTプロンプトをカスタマイズできます。

- 検索クエリをカスタマイズして、特定のSharePointサイトやO365ドライブのみを検索するようにコードをカスタマイズできます。これにより検索を集中させ、検索精度を向上させることができます。現在の設定では、ログインユーザーがアクセスできるすべてのファイルを検索します。

- 特定のタイプのファイルのみを返すようにコードを更新できます。例えば、構造化データ/CSVのみを返すなど。

- Microsoft Graphへの呼び出し内で検索するファイル数をカスタマイズできます。[こちら](https://platform.openai.com/docs/actions/getting-started)のドキュメントに基づいて、最大10ファイルのみにすべきであることに注意してください。

### 考慮事項

10万文字以下の返却と[45秒のタイムアウト](https://platform.openai.com/docs/actions/production/timeouts)に関して、Actionsの同じ制限がすべてここでも適用されることに注意してください。

- [ファイルの返却](https://platform.openai.com/docs/actions/sending-files)と[ファイルアップロード](https://help.openai.com/en/articles/8555545-file-uploads-faq)に関するドキュメントを必ず読んでください。これらの制限がここでも適用されます。

## ChatGPT ステップ

### カスタムGPT指示

Custom GPTを作成したら、以下のテキストをInstructionsパネルにコピーしてください。質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
You are a Q&A helper that helps answer users questions. You have access to a documents repository through your API action. When a user asks a question, you pass in the "searchTerm" a single keyword or term you think you should use for the search.

****

Scenario 1: There are answers

If your action returns results, then you take the results from the action and try to answer the users question. 

****

Scenario 2: No results found

If the response you get from the action is "No results found", stop there and let the user know there were no results and that you are going to try a different search term, and explain why. You must always let the user know before conducting another search.

Example:

****

I found no results for "DEI". I am now going to try [insert term] because [insert explanation]

****

Then, try a different searchTerm that is similar to the one you tried before, with a single word. 

Try this three times. After the third time, then let the user know you did not find any relevant documents to answer the question, and to check SharePoint. 
Be sure to be explicit about what you are searching for at each step.

****

In either scenario, try to answer the user's question. If you cannot answer the user's question based on the knowledge you find, let the user know and ask them to go check the HR Docs in SharePoint. 

### OpenAPI スキーマ

Custom GPTを作成したら、以下のテキストをActionsパネルにコピーしてください。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

これは、[こちら](https://platform.openai.com/docs/actions/sending-files)のドキュメントにあるファイル取得構造と一致するレスポンスを期待し、検索を通知するために`searchTerm`パラメータを渡します。
>上記のスクリーンショットでコピーされたリンクに基づいて、関数アプリ名、関数名、およびコードを必ず切り替えてください

In [ ]:
openapi: 3.1.0
info:
  title: SharePoint Search API
  description: API for searching SharePoint documents.
  version: 1.0.0
servers:
  - url: https://{your_function_app_name}.azurewebsites.net/api
    description: SharePoint Search API server
paths:
  /{your_function_name}?code={enter your specific endpoint id here}:
    post:
      operationId: searchSharePoint
      summary: Searches SharePoint for documents matching a query and term.
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                searchTerm:
                  type: string
                  description: A specific term to search for within the documents.
      responses:
        '200':
          description: A CSV file of query results encoded in base64.
          content:
            application/json:
              schema:
                type: object
                properties:
                  openaiFileResponseData:
                    type: array
                    items:
                      type: object
                      properties:
                        name:
                          type: string
                          description: The name of the file.
                        mime_type:
                          type: string
                          description: The MIME type of the file.
                        content:
                          type: string
                          format: byte
                          description: The base64 encoded contents of the file.
        '400':
          description: Bad request when the SQL query parameter is missing.
        '413':
          description: Payload too large if the response exceeds the size limit.
        '500':
          description: Server error when there are issues executing the query or encoding the results.

## 認証手順

以下は、このサードパーティアプリケーションでの認証設定に関する手順です。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

*認証に関するより詳細な手順については、上記および[Azure Function cookbook](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_middleware_azure_function)を参照してください。*

## FAQ & トラブルシューティング

- なぜコード内で[SharePoint API](https://learn.microsoft.com/en-us/sharepoint/dev/sp-add-ins/get-to-know-the-sharepoint-rest-service?tabs=csom)ではなくMicrosoft Graph APIを使用しているのですか？

  - SharePoint APIはレガシーです - Microsoftのドキュメント[こちら](https://learn.microsoft.com/en-us/sharepoint/dev/apis/sharepoint-rest-graph)によると、「SharePoint Onlineでは、SharePointに対するREST APIを使用したイノベーションは、Microsoft Graph REST APIを通じて推進されています。」Graph APIはより柔軟性を提供し、SharePoint APIでも[Microsoft Graph APIと直接やり取りする代わりに、なぜこれが必要なのですか？](#why-is-this-necessary-instead-of-interacting-with-the-microsoft-api-directly)セクションに記載されているのと同じファイルの問題が発生します。

- これはどのような種類のファイルをサポートしていますか？

  ファイルアップロードに関する[こちら](https://help.openai.com/en/articles/8555545-file-uploads-faq)のドキュメントと同じガイドラインに従います。

- なぜOBOトークンをリクエストする必要があるのですか？

  - Azure Functionへの認証に使用するのと同じトークンをGraph APIへの認証に使用しようとすると、「無効なオーディエンス」トークンエラーが発生します。これは、トークンのオーディエンスがuser\_impersonationのみに設定できるためです。

  - これに対処するため、この関数は[On Behalf Ofフロー](https://learn.microsoft.com/en-us/entra/identity-platform/v2-oauth2-on-behalf-of-flow)を使用してアプリ内でFiles.Read.Allにスコープされた新しいトークンをリクエストします。これによりログインユーザーの権限が継承され、この関数はログインユーザーがアクセス権を持つファイルのみを検索することになります。

  - Azure Function Appsはステートレスであることを意図しているため、各リクエストで意図的に新しいOn Behalf Ofトークンをリクエストしています。Azure Key Vaultと統合してシークレットを保存し、プログラム的に取得することも可能です。

*私たちに優先的に対応してほしい統合機能はありますか？統合機能にエラーがありますか？GitHubでPRやissueを作成していただければ、確認いたします。*